In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# The goal of this file is to show that for large datasets:
#  1) we can solve fair PCA or multi-criteria dimensionality reduction quickly.
# the objectives used are 1) variance 2) mar-loss (both are similar - just a constant shift), and 3) NSW
# The NSW does not fit into MW framework. However, the problem has quick linear oracle, namely SVD, so we will use Frank-Wolfe for NSW.

# 2) The heuristics have loose theory bound, but we can tune more aggressively and certify solutions by duality gap

from optimizer import fairDimReduction_MW, FW
from utils import getObj

In [3]:
B=[]
m=661967 #total number of datapoints
n=1000
k=16
group_sizes = np.load('group_sizes.npy')
for idx in range(k):
    #load data, use standard normalization on group size by equalizing importance of each group
    B.append(np.load('./mat/n='+str(n)+'_group'+str(idx+1)+'.npy')/group_sizes[idx]*1000)
    # times 1000 so the number is not too small for large normalization constant in large dataset

In [25]:
#experiment n=1000; Mar_loss by MW
d=3
#creating adaptive eta
eta = [4 for i in range(10)]
for j in range(100):
    eta = eta + [max(eta[-1]*0.8,2) for i in range(10)]
#do the MW
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,B,eta=2.5 #tuned to be 2.7 for static eta
                                      , T=400, verbose = False,Obj='MM_Loss',calculate_dual=True, n_X_last=24,stopping_gap=1.4e-4)
#Xs are several last iterates 
#Xavg is the average iterate
stats


#the run time shows that MW takes about 0.6 sec/iteration for n=1000,k=16.
# this one has an exact solution from the last iterate. With eta=2.5, gap=1e-4, it stops at T=142 iterates

MW terminated at T= 118  iterations: current iterate solution achieved primal-dual gap of 0.00014
fairDimReduction_MW is called. Total time used is:  77.71131609999975  seconds.
The best solution found from avg and single iterate acheieves primal -0.14150248285517664 . The dual is -0.141363164452153 . Gap is 0.00013931840302364162 which is 0.09855354014149602 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,-0.302987,-0.302987,-0.093048
1,1,-0.216194,-0.259591,-0.104292
2,2,-0.190379,-0.226033,-0.111062
3,3,-0.160166,-0.205193,-0.114703
4,4,-0.160357,-0.192186,-0.117092
5,5,-0.160229,-0.183104,-0.119297
6,6,-0.159881,-0.176267,-0.121337
7,7,-0.159384,-0.170885,-0.123230
8,8,-0.158823,-0.166594,-0.124987
9,9,-0.158186,-0.163150,-0.126619


In [7]:
#for n=1000, MM_loss, the last iterate is the optimal solution

#since it is the last iterate, it also has exaxct rank
sorted(np.linalg.eigh(Xs)[0],reverse=True)

[1.0000000000000013,
 0.9999999999999998,
 0.9999999999999979,
 6.526419219871455e-16,
 5.914390572201431e-16,
 4.3007829475639817e-16,
 2.630167598149304e-16,
 2.4946595212598515e-16,
 1.7413945350402935e-16,
 1.5872182617648402e-16,
 1.0780872101241593e-16,
 1.0735282810062058e-16,
 8.310128317517809e-17,
 6.004432239744871e-17,
 3.6174497754473094e-17,
 3.204867335153027e-17,
 2.7737845806861717e-17,
 2.5457763527931308e-17,
 2.4664544704282613e-17,
 2.236572732263492e-17,
 2.2127459841053205e-17,
 2.0051460050179386e-17,
 1.8622219564388308e-17,
 1.7822312480252305e-17,
 1.688405863469376e-17,
 1.5467568369331163e-17,
 1.4369760673093117e-17,
 1.4069538835655974e-17,
 1.2299232158664e-17,
 1.2136325286646103e-17,
 1.1492551509200873e-17,
 1.1440462140009351e-17,
 1.0557961232297456e-17,
 1.0154878044005431e-17,
 9.631612414377909e-18,
 8.893680779613478e-18,
 8.043051192615866e-18,
 7.956038814019482e-18,
 7.5744772395016e-18,
 6.6854885128895525e-18,
 6.316432014786616e-18,
 6.099

In [24]:
#experiment n=1000; MM_var by MW
d=3
#creating changing eta
eta = [20 for i in range(10)]
for j in range(100):
    eta = eta + [max(eta[-1]-0.5,15) for i in range(10)]
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,B,eta=15 #15 for tuned static value
                                      , T=400, verbose = False,Obj='MM_Var',calculate_dual=True, n_X_last=24,stopping_gap=5.5e-5)
stats

#the run time shows that MW takes about 0.6 sec/iteration for n=1000,k=16.
#use eta=15 and it finishes in 458 iterations, 294 seconds. Tuned with changing eta and it was 172 iterations

MW terminated at T= 99  iterations: current iterate solution achieved primal-dual gap of 5.5e-05
fairDimReduction_MW is called. Total time used is:  65.06115780000073  seconds.
The best solution found from avg and single iterate acheieves primal 0.05934995519313093 . The dual is 0.0594046910019137 . Gap is 5.4735808782770123e-05 which is 0.09214054960913243 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,0.037588,0.037588,0.072520
1,1,0.049790,0.045525,0.063131
2,2,0.051385,0.047479,0.061953
3,3,0.056179,0.049707,0.061201
4,4,0.056511,0.051068,0.060857
5,5,0.057042,0.052097,0.060620
6,6,0.057197,0.052885,0.060440
7,7,0.057205,0.053515,0.060294
8,8,0.057295,0.054031,0.060174
9,9,0.057427,0.054464,0.060072


In [37]:
print('Eig are',sorted(np.linalg.eigh(Xs)[0],reverse=True))
#pick last 3 vectors, which are ones with eigenvalues 1
P_guess = np.linalg.eigh(Xs[-1])[1][:,-3:]
X_guess = P_guess @ P_guess.T
getObj(n,k,d,B,X_guess)

Eig are [1.0000000000000009, 1.0, 0.9999999999999992, 5.309337539440776e-16, 5.062096459962405e-16, 4.1097604634503368e-16, 4.001226049751107e-16, 2.6165412903748514e-16, 2.3366169380511147e-16, 1.5886307777008864e-16, 1.412995938882325e-16, 1.1924670743707722e-16, 1.0736805512302619e-16, 7.736825531308836e-17, 5.199367689510368e-17, 4.369536312510203e-17, 3.092676545711667e-17, 2.873122342699599e-17, 2.858169373676102e-17, 2.7635555276777682e-17, 2.4365884712157786e-17, 2.401474764532203e-17, 2.195099969262068e-17, 2.1419339037160377e-17, 2.0540389276026985e-17, 1.9732375872275023e-17, 1.9275340974308863e-17, 1.7697279534033273e-17, 1.662938044582551e-17, 1.64220411083069e-17, 1.619987023331993e-17, 1.526855359103298e-17, 1.407692587371086e-17, 1.3154232171716432e-17, 1.287035156464799e-17, 1.2803828915743304e-17, 1.2554428241856691e-17, 1.193822722982326e-17, 1.1357657030414717e-17, 1.0829004740899357e-17, 1.0556854428873373e-17, 1.0505093291643556e-17, 1.0307700648440805e-17, 9.6967

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [26]:
#experiment n=1000; NSW by FW
d=3
X = FW(n,k,d,B,Obj='NSW',num_iterations=1000,update_rule='1/2')  

#for n=1000,k=16,d=3 FW takes about 0.4 sec/iterations for easy update rule and NSW objective. 

Iterations t=1: primal is -54.45225112073853, dual is 314.605819073732, and gap is 369.05807019447053 which is 1.1730808771467032%. Time taken: 0.7469529000009061 seconds.
Iterations t=2: primal is -47.79907948184656, dual is -30.196155418561233, and gap is 17.60292406328533 which is 0.5829524924376634%. Time taken: 1.5219457000002876 seconds.
Iterations t=3: primal is -45.25262313363766, dual is -39.67688369184895, and gap is 5.575739441788711 which is 0.1405286636191685%. Time taken: 2.276805299999978 seconds.
Iterations t=4: primal is -44.11058253175177, dual is -41.73802274942417, and gap is 2.3725597823276017 which is 0.056844086663408885%. Time taken: 3.005010000000766 seconds.
Iterations t=5: primal is -43.568176615043264, dual is -42.46580817456398, and gap is 1.102368440479283 which is 0.025958965291506556%. Time taken: 3.7398239000012836 seconds.
Iterations t=6: primal is -43.30365121312946, dual is -42.77096920365077, and gap is 0.5326820094786907 which is 0.0124542889580632

In [4]:
#experiment n=1000; Let's try FW on MM_Loss too
d=3
X = FW(n,k,d,B,Obj='MM_Loss',num_iterations=1000,update_rule='1/2_every_10')  

#for n=1000,k=16,d=3 FW takes about 0.4 sec/iterations for easy update rule and NSW objective. 

sanity check: sum of w is 0.9999999999999998 w is [5.468475137883213e-07, 9.012673972803508e-08, 5.856837205498987e-08, 7.005478860402142e-08, 3.90670141272631e-07, 1.050227162509125e-05, 0.9990695132731712, 0.0007273736958614727, 6.336207335190709e-07, 1.2871989784511757e-07, 2.0746714475933803e-08, 3.7984981195043375e-08, 7.406198272956235e-08, 2.2997607620862315e-06, 0.00014797016328285667, 4.028943343192884e-05]
Iterations t=1: primal is -0.2746541790126207, dual is 0.15022377163264167, and gap is 0.42487795064526235 which is 2.828300381675026%. Time taken: 3.0843858 seconds.
sanity check: sum of w is 0.9999999999999999 w is [0.000787806080991685, 0.00013488982994309425, 8.713913348403304e-05, 0.00010264110112809946, 0.0005210581351828687, 0.008456341794738555, 0.037995177789807, 0.8445367002967915, 0.0008939400546824894, 0.00018853370867299307, 3.171774264107311e-05, 5.870388307733407e-05, 0.00011301665702009691, 0.0034947941610997354, 0.05149446236159806, 0.0511030772691411]
Iter

Iterations t=15: primal is -0.1829093994574872, dual is -0.14194221773952762, and gap is 0.04096718171795957 which is 0.288618723663574%. Time taken: 46.617833999999995 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.01211214785681218, 0.004035199398457707, 0.002652923860939012, 0.0033060892964263573, 0.013667082979325635, 0.15542271096207294, 0.48689464108357366, 0.03961185863775197, 0.013954534875321244, 0.006019230613235076, 0.0011813259865564312, 0.002375722855961371, 0.004055798619258089, 0.08536762727998984, 0.12546958294728036, 0.043873522747038284]
Iterations t=16: primal is -0.16605572417013717, dual is -0.09116484556012065, and gap is 0.07489087861001652 which is 0.8214885699622898%. Time taken: 49.6231041 seconds.
sanity check: sum of w is 1.0 w is [0.01861606416730572, 0.005392748669109303, 0.0035204614798411337, 0.004283797303706529, 0.018013897159064096, 0.19918585968926047, 0.06366815735505688, 0.24948782678179404, 0.021202307964231924, 0.007865944269281092

Iterations t=30: primal is -0.155737884857876, dual is -0.1345877527019819, and gap is 0.02115013215589413 which is 0.15714752443134203%. Time taken: 94.1337874 seconds.
sanity check: sum of w is 0.9999999999999999 w is [0.01959229981802212, 0.006549664661751562, 0.004238592753370485, 0.005139851248892956, 0.020431496465775162, 0.20643656853132555, 0.11916818389600548, 0.19423497597733716, 0.02221027968517623, 0.009683991051292152, 0.0019312789527061848, 0.0038538227508027835, 0.006532944286815707, 0.13874313268135555, 0.1313601370436719, 0.10989278019569897]
Iterations t=31: primal is -0.1555191271654146, dual is -0.13922682005299958, and gap is 0.016292307112415014 which is 0.11701988960326051%. Time taken: 97.11875280000001 seconds.
sanity check: sum of w is 1.0 w is [0.01899578578319866, 0.006543910381078073, 0.004237964354356892, 0.005174001071304997, 0.02044636628135796, 0.20943771155849714, 0.18237468530654086, 0.13675306807728974, 0.021571143340173123, 0.009724383468598859, 0.0

Iterations t=45: primal is -0.15227077069354034, dual is -0.14829911900680934, and gap is 0.003971651686730994 which is 0.026781357255053113%. Time taken: 139.50077339999999 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02113850544494967, 0.007962888047829431, 0.004972453077077835, 0.00596474138212986, 0.02322077459580863, 0.22089670368364664, 0.18555718729118914, 0.13904982750632133, 0.023835253021619474, 0.01192702415816244, 0.002408296124816469, 0.0047623592176609325, 0.007978611353675702, 0.15653402482372775, 0.11260810534840454, 0.07118324492298024]
Iterations t=46: primal is -0.15212232790242822, dual is -0.14827696269963037, and gap is 0.0038453652027978613 which is 0.025933665842532375%. Time taken: 142.4737844 seconds.
sanity check: sum of w is 1.0 w is [0.02120054246052599, 0.008000717966717637, 0.0049910521572621, 0.005984192187936244, 0.02327679236050295, 0.22058427356457797, 0.18551127048666002, 0.13899192255099943, 0.023901947299906186, 0.01198674697144983

Iterations t=60: primal is -0.1509692010655462, dual is -0.1481336380084806, and gap is 0.0028355630570656054 which is 0.01914192546127356%. Time taken: 184.6202568 seconds.
sanity check: sum of w is 1.0 w is [0.021667459063596795, 0.008291103771485788, 0.005133138092360984, 0.006132596434881739, 0.023692945454241412, 0.21802861634905463, 0.18517048066325856, 0.13852351094862086, 0.024406163973939528, 0.012447192327766702, 0.002522271326255916, 0.004975453587128367, 0.00830662503346206, 0.15909842940933655, 0.11184037566487606, 0.06976363789973401]
Iterations t=61: primal is -0.1509416810703605, dual is -0.14815090062218056, and gap is 0.0027907804481799234 which is 0.018837418041062558%. Time taken: 187.54317329999998 seconds.
sanity check: sum of w is 0.9999999999999999 w is [0.021678255675861107, 0.008297999373824904, 0.00513649961463584, 0.006136104329998957, 0.023702549780544504, 0.21796629158170072, 0.18516251121995184, 0.1385117502359705, 0.024417870885136927, 0.0124581685335294

...

sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=932: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2643.7429463 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=933: primal is

Iterations t=946: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2682.5349558 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=947: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2685.2858369 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.

sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=961: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2723.4180827 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=962: primal is

Iterations t=975: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2761.4038373 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=976: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2764.073265 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.0

sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=990: primal is -0.15045851858897852, dual is -0.1480760130987207, and gap is 0.002382505490257813 which is 0.016089746343112457%. Time taken: 2802.5802554 seconds.
sanity check: sum of w is 1.0000000000000002 w is [0.02186487645040739, 0.008419129923324845, 0.0051954701468122306, 0.006197621436957128, 0.023869325359059484, 0.21687269525131206, 0.18502213605999843, 0.13830021910345477, 0.024620556316505483, 0.012651278569783004, 0.0025673883223308936, 0.005059768413753156, 0.008434746308779807, 0.15999569480234438, 0.11156853398478012, 0.06936055955039694]
Iterations t=991: primal is

In [9]:
sorted(np.linalg.eigh(X)[0],reverse=True) #the solution seems to have an exact rank

[0.6289710623660332,
 0.5236581016944067,
 0.494633016219261,
 0.3487226796921616,
 0.34476656959428376,
 0.29993093943289506,
 0.15063559628138287,
 0.1316078426069283,
 0.07706207972902851,
 1.2222385431928896e-08,
 1.2222385294527974e-08,
 1.2222385241116702e-08,
 1.2222385228542112e-08,
 1.2222385195400228e-08,
 1.2222385190935234e-08,
 1.2222385177072862e-08,
 1.22223851760397e-08,
 1.2222385163337227e-08,
 1.2222385159318406e-08,
 1.2222385154083383e-08,
 1.222238514366041e-08,
 1.2222385130681628e-08,
 1.2222385127896124e-08,
 1.2222385124888889e-08,
 1.2222385121915765e-08,
 1.222238511875116e-08,
 1.2222385116908787e-08,
 1.222238511235643e-08,
 1.22223851118378e-08,
 1.222238510992125e-08,
 1.2222385108481522e-08,
 1.2222385104541857e-08,
 1.2222385104325295e-08,
 1.2222385104238522e-08,
 1.222238510250619e-08,
 1.2222385100079177e-08,
 1.2222385100019401e-08,
 1.2222385099564576e-08,
 1.2222385099548704e-08,
 1.2222385097353299e-08,
 1.2222385096892837e-08,
 1.22223850959026

In [5]:
#experiment n=1000; Let's try FW on MM_var too
d=3
X = FW(n,k,d,B,Obj='MM_Var',num_iterations=1000,update_rule='1/2_every_10')  

#for n=1000,k=16,d=3 FW takes about 0.4 sec/iterations for easy update rule and NSW objective. 

Iterations t=1: primal is 0.009187668124199756, dual is 0.1490251786953731, and gap is 0.13983751057117336 which is 0.9383482160220687%. Time taken: 0.26506149999977424 seconds.
Iterations t=2: primal is 0.02000820492522668, dual is 0.110426509450473, and gap is 0.09041830452524632 which is 0.8188097674662035%. Time taken: 0.5864399000001868 seconds.
Iterations t=3: primal is 0.025832326118538078, dual is 0.10155310147158461, and gap is 0.07572077535304653 which is 0.7456274033563989%. Time taken: 0.9611085999999887 seconds.
Iterations t=4: primal is 0.03305182050668373, dual is 0.08120824789101869, and gap is 0.04815642738433495 which is 0.592999216643127%. Time taken: 1.3451058999999077 seconds.
Iterations t=5: primal is 0.020373004547246262, dual is 0.21264260169067944, and gap is 0.19226959714343317 which is 0.9041913314393987%. Time taken: 1.689974699999766 seconds.
Iterations t=6: primal is 0.017261635429453182, dual is 0.10017378270705425, and gap is 0.08291214727760107 which is

Iterations t=48: primal is 0.036252135711203726, dual is 0.07406126704075441, and gap is 0.03780913132955069 which is 0.5105115378156451%. Time taken: 16.59040899999991 seconds.
Iterations t=49: primal is 0.03655578063260177, dual is 0.14193034557768974, and gap is 0.10537456494508797 which is 0.7424385850410553%. Time taken: 16.941267000000153 seconds.
Iterations t=50: primal is 0.03664065830516399, dual is 0.08630489315455353, and gap is 0.04966423484938954 which is 0.5754509742622769%. Time taken: 17.274750900000072 seconds.
Iterations t=51: primal is 0.03670744331440334, dual is 0.10335193683408674, and gap is 0.06664449351968339 which is 0.6448306201234462%. Time taken: 17.650425799999994 seconds.
Iterations t=52: primal is 0.03696429734435963, dual is 0.07424560753282936, and gap is 0.03728131018846972 which is 0.5021348929225875%. Time taken: 17.994567199999892 seconds.
Iterations t=53: primal is 0.03703259933868309, dual is 0.09579728227259662, and gap is 0.058764682933913535 w

Iterations t=95: primal is 0.038363407735310946, dual is 0.11001606444019432, and gap is 0.07165265670488338 which is 0.6512926732062333%. Time taken: 32.671533700000055 seconds.
Iterations t=96: primal is 0.038375641934942785, dual is 0.07400098770250488, and gap is 0.0356253457675621 which is 0.48141716581920974%. Time taken: 32.98365130000002 seconds.
Iterations t=97: primal is 0.03836982670035939, dual is 0.13876815577544163, and gap is 0.10039832907508224 which is 0.7234968895713331%. Time taken: 33.32998150000003 seconds.
Iterations t=98: primal is 0.0383637209923038, dual is 0.09572287457021758, and gap is 0.057359153577913775 which is 0.5992209681901888%. Time taken: 33.68921760000012 seconds.
Iterations t=99: primal is 0.03838640185847976, dual is 0.074011434369049, and gap is 0.03562503251056925 which is 0.4813449815460859%. Time taken: 34.024102400000174 seconds.
Iterations t=100: primal is 0.03840531166813675, dual is 0.09962488245910309, and gap is 0.061219570790966335 whi

Iterations t=141: primal is 0.03845329924552407, dual is 0.0739898331198304, and gap is 0.035536533874306324 which is 0.4802894178278934%. Time taken: 48.294543800000156 seconds.
Iterations t=142: primal is 0.038452705334961416, dual is 0.1100298325534281, and gap is 0.0715771272184667 which is 0.6505247309515843%. Time taken: 48.6638210000001 seconds.
Iterations t=143: primal is 0.03845378980908766, dual is 0.07398983797699926, and gap is 0.0355360481679116 which is 0.4802828217972103%. Time taken: 48.99585789999992 seconds.
Iterations t=144: primal is 0.038454598821373163, dual is 0.07398956251515855, and gap is 0.035534963693785386 which is 0.48026995275860956%. Time taken: 49.3318905000001 seconds.
Iterations t=145: primal is 0.038454777372791356, dual is 0.09960615120086427, and gap is 0.06115137382807291 which is 0.6139317009122858%. Time taken: 49.67954400000008 seconds.
Iterations t=146: primal is 0.038454658089701325, dual is 0.09572886099518316, and gap is 0.05727420290548183

Iterations t=187: primal is 0.038457056713920953, dual is 0.08622001647385544, and gap is 0.04776295975993449 which is 0.5539660245184213%. Time taken: 63.869634600000154 seconds.
Iterations t=188: primal is 0.03845704432399952, dual is 0.11003041407406934, and gap is 0.07157336975006982 which is 0.6504871435082364%. Time taken: 64.23784539999997 seconds.
Iterations t=189: primal is 0.0384570656931818, dual is 0.07398877487205532, and gap is 0.03553170917887352 which is 0.48023107884022315%. Time taken: 64.55884650000007 seconds.
Iterations t=190: primal is 0.03845705396725784, dual is 0.1387739592841012, and gap is 0.10031690531684334 which is 0.7228798964470871%. Time taken: 64.86592310000015 seconds.
Iterations t=191: primal is 0.038457061320836254, dual is 0.09572898763185161, and gap is 0.05727192631101537 which is 0.5982715134444757%. Time taken: 65.23532710000018 seconds.
Iterations t=192: primal is 0.03845709223960942, dual is 0.07398878442164616, and gap is 0.03553169218203675

...

Iterations t=792: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 272.6428170999998 seconds.
Iterations t=793: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 272.95605909999995 seconds.
Iterations t=794: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 273.3264254999999 seconds.
Iterations t=795: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 273.7101115999999 seconds.
Iterations t=796: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 274.01995769999985 seconds.
Iterations t=797: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which

Iterations t=839: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 288.6219769999998 seconds.
Iterations t=840: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 288.97089559999995 seconds.
Iterations t=841: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 289.32565209999984 seconds.
Iterations t=842: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 289.65623189999997 seconds.
Iterations t=843: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 289.98815049999985 seconds.
Iterations t=844: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 whi

Iterations t=886: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 305.0179615000002 seconds.
Iterations t=887: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 305.34626660000004 seconds.
Iterations t=888: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 305.6924374 seconds.
Iterations t=889: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 306.0198105999998 seconds.
Iterations t=890: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 306.3550574000001 seconds.
Iterations t=891: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4

Iterations t=933: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 320.9640395000001 seconds.
Iterations t=934: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 321.29635229999985 seconds.
Iterations t=935: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 321.64384329999984 seconds.
Iterations t=936: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 321.9672096999998 seconds.
Iterations t=937: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 322.3255436999998 seconds.
Iterations t=938: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which

Iterations t=980: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 338.23025929999994 seconds.
Iterations t=981: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 338.5629981000002 seconds.
Iterations t=982: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 338.89889140000014 seconds.
Iterations t=983: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 339.2323290999998 seconds.
Iterations t=984: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which is 0.4802289033415804%. Time taken: 339.5591122999999 seconds.
Iterations t=985: primal is 0.0384572155485778, dual is 0.07398875350287303, and gap is 0.03553153795429523 which

Next, we do three experiments MM_loss, MM_var, and NSW on larger dimensions n=2000.

In [27]:
C=[]
m=661967 #total number of datapoints
n=2000
k=16
group_sizes = np.load('group_sizes.npy')
for idx in range(k):
    #load data, use standard normalization on group size by equalizing importance of each group
    C.append(np.load('./mat/n='+str(n)+'_group'+str(idx+1)+'.npy')/group_sizes[idx]*1000)
    # times 1000 so the number is not too small for large normalization constant in large dataset

In [28]:
#experiment n=2000; Mar_loss by MW
d=3
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,C,eta=3, T=1000, verbose = False,Obj='MM_Loss',calculate_dual=True, n_X_last=24,stopping_gap=1.4e-4)
stats

#the run time shows that MW with eta=3 takes about 3.3 sec/iteration for n=2000,k=16. 5 times longer than n=1000.
#total time about 3000 seconds or 50 minutes.

MW terminated at T= 197  iterations: current iterate solution achieved primal-dual gap of 0.00014
fairDimReduction_MW is called. Total time used is:  588.9505711000002  seconds.
The best solution found from avg and single iterate acheieves primal -0.1482778617047387 . The dual is -0.14813851031809006 . Gap is 0.00013935138664863955 which is 0.09406830563465073 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,-0.302058,-0.302058,-0.099925
1,1,-0.215791,-0.258925,-0.113424
2,2,-0.183923,-0.220667,-0.120553
3,3,-0.168547,-0.202012,-0.123967
4,4,-0.161478,-0.190985,-0.126880
5,5,-0.161678,-0.183576,-0.129423
6,6,-0.161772,-0.178179,-0.131665
7,7,-0.161715,-0.174045,-0.133649
8,8,-0.161508,-0.170769,-0.135406
9,9,-0.161166,-0.168112,-0.136961


In [29]:
#experiment n=2000; MM_var by MW
d=3
[Xs,Xavg,stats] = fairDimReduction_MW(n,k,d,C,eta=11, T=2000, verbose = False,Obj='MM_Var',calculate_dual=True, n_X_last=24,stopping_gap=5.5e-5)
stats

#the run time shows that MW eith eta=11 takes about 3.3 sec/iteration for n=2000,k=16. 5 times longer than n=1000

MW terminated at T= 213  iterations: current iterate solution achieved primal-dual gap of 5.5e-05
fairDimReduction_MW is called. Total time used is:  585.4562041999998  seconds.
The best solution found from avg and single iterate acheieves primal 0.06155820278322715 . The dual is 0.06161314834004326 . Gap is 5.49455568161078e-05 which is 0.08917829764657215 %.


,iteration,"minimum of m objective, that iterate","minimum of m objective, avg iterate",dual objective
0,0,0.037671,0.037671,0.074670
1,1,0.049961,0.043816,0.066006
2,2,0.053849,0.047160,0.064733
3,3,0.055659,0.049285,0.064078
4,4,0.056906,0.050809,0.063666
5,5,0.057810,0.051976,0.063368
6,6,0.058500,0.052908,0.063134
7,7,0.058987,0.053674,0.062943
8,8,0.059176,0.054318,0.062783
9,9,0.059258,0.054868,0.062646


In [11]:
#experiment n=2000; NSW by FW
d=3
X = FW_NSW(n,k,d,C,num_iterations=2000,update_rule='1/2_plus_every_10')   #update_rule='1/2_plus_every_10' is the best

#for n=2000,k=16,d=3 FW takes about 2.4 sec/iterations. 6 times longer than n=1000. Total time is 68 secodns

Iterations t=1: gap is 567.3989177873624. Time taken: 2.6074320999999827 seconds.
Iterations t=2: gap is 17.267923541809644. Time taken: 5.901472199999944 seconds.
Iterations t=3: gap is 5.642016448902536. Time taken: 8.394656299999951 seconds.
Iterations t=4: gap is 2.3641087281162476. Time taken: 10.86945490000005 seconds.
Iterations t=5: gap is 1.107104589234063. Time taken: 13.359743200000025 seconds.
Iterations t=6: gap is 0.542752014245574. Time taken: 15.861350900000048 seconds.
Iterations t=7: gap is 0.2757225305399501. Time taken: 18.194097400000032 seconds.
Iterations t=8: gap is 0.14426842739690537. Time taken: 20.885123600000043 seconds.
Iterations t=9: gap is 0.08021346814868753. Time taken: 23.450990000000047 seconds.
Iterations t=10: gap is 0.04715510532777105. Time taken: 26.293430400000034 seconds.
Iterations t=11: gap is 0.031632446048617736. Time taken: 28.82434490000003 seconds.
Iterations t=12: gap is 0.017081301750489226. Time taken: 31.446317099999987 seconds.
It

In [30]:
sorted(np.linalg.eigh(X)[0],reverse=True) #exact rank solution found

[0.9999980475700205,
 0.999997804338386,
 0.9999957104732975,
 2.314036758849733e-06,
 2.773824872395951e-07,
 1.0055119650686998e-07,
 5.230593163284879e-08,
 1.0153343860720396e-08,
 9.070423059666659e-09,
 8.233812473662934e-09,
 6.3837232542658405e-09,
 5.940131260358521e-09,
 5.8215815388611955e-09,
 5.769709455136373e-09,
 5.738986692854279e-09,
 5.734513508623118e-09,
 5.723748645347344e-09,
 5.722365553202477e-09,
 5.722273189010097e-09,
 5.722244296532333e-09,
 5.7221134447412585e-09,
 5.722060359999378e-09,
 5.722046653832653e-09,
 5.722046128148603e-09,
 5.722046105084036e-09,
 5.7220460740661545e-09,
 5.722046065191774e-09,
 5.722046044966854e-09,
 5.7220460435800284e-09,
 5.722046010913335e-09,
 5.722045989022031e-09,
 5.722045971368563e-09,
 5.7220459643916775e-09,
 5.722045962979465e-09,
 5.722045951603119e-09,
 5.722045948019013e-09,
 5.722045942763104e-09,
 5.722045940133739e-09,
 5.722045932166854e-09,
 5.72204593108737e-09,
 5.7220459300249055e-09,
 5.722045929099166

In [9]:
#experiment n=2000; NSW by FW
d=3
X = FW_NSW(n,k,d,C,num_iterations=2000,update_rule='line_search')   #update_rule='1/2_plus_every_10' is the best

#for n=2000,k=16,d=3 FW takes about 13 sec/iterations. Final time is 172 seconds. 

Iterations t=1: gap is 567.3989177873624. Time taken: 13.261866199999986 seconds.
Iterations t=2: gap is 1.361810655810486. Time taken: 26.323872700000038 seconds.
Iterations t=3: gap is 0.41717768593513527. Time taken: 39.40350670000004 seconds.
Iterations t=4: gap is 0.1634066388412906. Time taken: 52.12746880000003 seconds.
Iterations t=5: gap is 0.03286970105800399. Time taken: 64.91284519999999 seconds.
Iterations t=6: gap is 0.03197765532023954. Time taken: 77.74000870000003 seconds.
Iterations t=7: gap is 0.006866365796487945. Time taken: 90.8830898 seconds.
Iterations t=8: gap is 0.004287625551797228. Time taken: 104.41399490000003 seconds.
Iterations t=9: gap is 0.004094327675558213. Time taken: 117.5634521 seconds.
Iterations t=10: gap is 0.0017538687822133833. Time taken: 132.32470090000004 seconds.
Iterations t=11: gap is 0.000373586511452397. Time taken: 146.8648549 seconds.
Iterations t=12: gap is 0.00010257557289670325. Time taken: 159.6624261 seconds.
Iterations t=13: g

In [10]:
sorted(np.linalg.eigh(X)[0],reverse=True)
# The final matrix from line_search, however, has more accuracy (eigenvalues closer to 1 up to more digits)

[0.9999999983079898,
 0.9999999974034789,
 0.9999974769402817,
 2.5167896171293188e-06,
 6.219438181080285e-09,
 2.5013742267988665e-09,
 1.402381363391363e-09,
 3.407685419252061e-10,
 4.06486539329667e-11,
 3.846957664932723e-11,
 1.1750045029509428e-11,
 2.3223141921776874e-12,
 8.046180223655478e-13,
 2.728770685733577e-13,
 1.4754369414607381e-13,
 6.211576844289792e-14,
 4.4151152005916975e-14,
 2.7314656816646464e-14,
 1.4110363885862601e-14,
 1.021171976953225e-14,
 9.82345205309398e-15,
 5.050023970130361e-15,
 3.698767400564163e-15,
 1.973286460212927e-15,
 1.815743037736269e-15,
 1.595552309280397e-15,
 1.0501292118433428e-15,
 6.768877883081162e-16,
 5.419440027641034e-16,
 5.227738119777065e-16,
 4.827325469126696e-16,
 3.8783146883145656e-16,
 1.9579665787208583e-16,
 1.7007270636402124e-16,
 1.6133365063252993e-16,
 1.5778557834631594e-16,
 1.5093992104773555e-16,
 1.3423980396927027e-16,
 1.3102339547381166e-16,
 1.2636260678782973e-16,
 1.1421651458080863e-16,
 1.08424